In [1]:
# Importando os Pacotes
# Manipulação de dados
import pandas as pd
import numpy as np
import datetime as dt

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Estatística
import scipy
from scipy.stats import normaltest
from scipy.stats import chi2_contingency

# Ignore Warning
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")


In [2]:
# Carregando Dados
vendas = pd.read_excel('fato_vendas.xlsx')
vendedor = pd.read_excel('dim_vendedor.xlsx')
produtos = pd.read_excel('dim_produtos.xlsx')
fam_produtos = pd.read_excel('dim_familia_produtos.xlsx')

In [3]:
vendas.shape

(196158, 9)

In [4]:
# Análise Exploratória do Dataframe Vendas
indice_linha_em_branco = vendas[vendas['codigo_cliente'].isnull()].index[0]

linha_em_branco = vendas.iloc[indice_linha_em_branco]
linha_em_branco

codigo_contrato                    NF 111897
data_venda               2022-01-19 00:00:00
codigo_produto                       2730100
filial_venda                              17
quantidade                                 1
valor_monetario_total                   22.9
valor_desconto                           0.0
codigo_vendedor                          438
codigo_cliente                           NaN
Name: 186023, dtype: object

In [5]:
# Deletando linha em branco, pois não foi possível descobrir qual codigo do clinte 
vendas = vendas.drop(indice_linha_em_branco)

In [6]:
vendas.shape

(196157, 9)

In [7]:
# Vendas Duplicatas
# Use loc para exibir apenas as linhas duplicadas no DataFrame original
vendas_duplicatas = vendas.loc[vendas.duplicated(keep=False)]

vendas_duplicatas.head(6)

,codigo_contrato,data_venda,codigo_produto,filial_venda,quantidade,valor_monetario_total,valor_desconto,codigo_vendedor,codigo_cliente
1674,2563961,2022-01-05,504000,17,5,36.25,0.0,507,3974.0
1675,2563961,2022-01-05,504000,17,5,36.25,0.0,507,3974.0
7311,2573000,2022-01-13,1360100,1,1,35.70,0.0,615,763279.0
7312,2573000,2022-01-13,1360100,1,1,35.70,0.0,615,763279.0
7670,2573495,2022-01-13,2881702,14,1,29.90,0.0,589,341769.0
7671,2573495,2022-01-13,2881702,14,1,29.90,0.0,589,341769.0


In [8]:
vendas.drop_duplicates(keep = 'last', inplace = True)

In [9]:
vendas.shape

(196109, 9)

In [10]:
vendedor.shape

(50, 2)

In [11]:
produtos.shape

(4472, 6)

In [12]:
fam_produtos.shape

(75, 2)

### Pergunta 1: Será muito importante visualizar nesse relatório qual foi o total vendido no ano, Assim como as vendas ao longo do tempo, tanto por mês quanto por semana.

> A. Dica: Utilize Cartões para mostrar valores únicos e gráficos de linha para exibir tendências ao longo do tempo.

> B. Dica: Adicione filtros em seu relatório para que o gerente possa visualizar esses gráficos com diferentes contextos.

In [13]:
vendas['data_venda'].value_counts()

2022-01-06    1057
2022-01-10    1027
2022-01-14    1013
2022-01-07     993
2022-01-11     940
              ... 
2022-03-01     163
2022-12-31     155
2022-12-11       1
2022-10-23       1
2022-10-12       1
Name: data_venda, Length: 308, dtype: int64

In [14]:
vendas['valor_monetario_total'].sum()

24776965.38

#### Mês

In [15]:
vendas['mes'] = vendas['data_venda'].dt.month

In [16]:
vendas['mes'].value_counts()

1     20003
8     17592
2     17498
5     16351
3     16031
9     15938
7     15881
6     15610
12    15471
11    15398
4     15230
10    15106
Name: mes, dtype: int64

In [17]:
agrupado_por_mes = vendas.groupby('mes')['valor_monetario_total'].agg(['count', 'sum']).reset_index()

In [18]:
agrupado_por_mes = agrupado_por_mes.rename(columns={'count': 'quantidade_vendas', 'sum': 'valor_monetario_total'})

In [19]:
agrupado_por_mes

,mes,quantidade_vendas,valor_monetario_total
0,1,20003,1945669.17
1,2,17498,1822792.89
2,3,16031,1876835.58
3,4,15230,1976954.11
4,5,16351,2114548.53
5,6,15610,1935414.31
6,7,15881,2109419.87
7,8,17592,2458462.87
8,9,15938,2156000.77
9,10,15106,2139841.11


In [73]:
fig = go.Figure()

# Adicionando a barra para a quantidade de vendas
fig.add_trace(go.Bar(
    x=agrupado_por_mes['mes'],
    y=agrupado_por_mes['quantidade_vendas'],
    name='Quantidade Vendida',
    marker_color='indigo'
))

# Adicionando a linha para o valor monetário total
fig.add_trace(go.Scatter(
    x=agrupado_por_mes['mes'],
    y=agrupado_por_mes['valor_monetario_total'],
    name='Faturamento Mensal',
    mode='lines+markers',
    yaxis='y2',
    line=dict(width=3)
))

# Ajustando o layout
fig.update_layout(
    title='Quantidade de Vendas e Valor Monetário por Mês',
    xaxis=dict(
        title='Mês',
        tickvals=agrupado_por_mes['mes'],
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    ),
    yaxis=dict(title='Quantidade de Vendas'),
    yaxis2=dict(
        title='Faturamento Mensal',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        y=1.2,  # Ajuste o valor de 'y' para mover a legenda para cima
        x=1
    ),
    height=500,
    width=1000
)
fig.show()

#### Semana

In [21]:
vendas['semana'] = vendas['data_venda'].dt.isocalendar().week


# In[41]:


vendas['semana'].value_counts()


# In[42]:


# Agrupe por mês e calcule a contagem de vendas e o valor monetário total
agrupado_por_semana = vendas.groupby('semana')['valor_monetario_total'].agg(['count', 'sum']).reset_index()

# Renomeie as colunas para maior clareza
agrupado_por_semana = agrupado_por_semana.rename(columns={'count': 'quantidade_vendas', 'sum': 'valor_monetario_total'})

print(agrupado_por_semana)

    semana  quantidade_vendas  valor_monetario_total
0        1               4904              535718.08
1        2               5154              452688.17
2        3               4574              430156.16
3        4               4458              443019.02
4        5               4714              444062.76
5        6               4509              468302.03
6        7               4499              492151.68
7        8               4152              440799.32
8        9               3344              370476.65
9       10               4128              455317.96
10      11               3392              410014.88
11      12               3044              395473.38
12      13               3736              444229.51
13      14               3792              466636.90
14      15               3226              412124.99
15      16               3459              471108.66
16      17               3677              489971.60
17      18               3719              497

In [22]:

fig1 = go.Figure(go.Scatter(
    x=agrupado_por_semana['semana'],
    y=agrupado_por_semana['valor_monetario_total'],
    mode='lines+markers',
    line=dict(color='indigo'),
    marker=dict(color='indigo'),
))

# Adicionando rótulos e ajustando o layout
fig1.update_layout(
    title='Valor Total de Vendas por Semana do Ano',
    xaxis=dict(
        title='Semanas do Ano',
        tickmode='array',
        tickvals=list(range(1, 53)),  # Gera uma lista de 1 a 52 para as semanas
        ticktext=[str(i) for i in range(1, 53)]  # Rótulos de 1 a 52 como strings
    ),
    yaxis=dict(title='Valor Total de Vendas'),
    height=600,
    width=1050,
)

# Exibindo o gráfico
fig1.show()

### Pergunta 3: O gerente também precisa monitorar a performance dos seus vendedores. Para tanto, ele solicitou que você propusesse três indicadores de performance de vendase que você mensurasse os mesmos por vendedor neste relatório.

> A. Quais indicadores você escolheu e por quê?

> B. A partir dos indicadores que você propôs, aponte quais são os 5 melhores vendedores no ano de 2022.

> C. Que tratativa você sugeriria ao gerente para aumentar as vendas dos demais vendedores? Justifique sua resposta.

In [23]:
# Merge entre vendas e produtos usando 'codigo_produto'
vendas_produtos = pd.merge(vendas, produtos, on='codigo_produto')

# Merge entre vendas_produtos e fam_produtos usando 'codigo_familia'
resultado_final = pd.merge(vendas_produtos, fam_produtos, on='codigo_familia')

In [24]:
resultado_final.head()

,codigo_contrato,data_venda,codigo_produto,filial_venda,quantidade,valor_monetario_total,valor_desconto,codigo_vendedor,codigo_cliente,mes,semana,descricaoproduto,codigo_familia,preco_venda_unitario,custo_produto_unitario,lucro,descricaofamilia
0,2561471,2022-01-03,2767200,14,5,49.5,0.0,589,325938.0,1,1,Eco-Friendly Tires and Wheels Teal #409,DMR,8.0,3.3,4.7,Tires and Wheels
1,2561475,2022-01-03,2767200,10,12,118.8,0.0,511,181844.0,1,1,Eco-Friendly Tires and Wheels Teal #409,DMR,8.0,3.3,4.7,Tires and Wheels
2,2561505,2022-01-03,2767200,17,1,9.9,0.0,507,690852.0,1,1,Eco-Friendly Tires and Wheels Teal #409,DMR,8.0,3.3,4.7,Tires and Wheels
3,2561521,2022-01-03,2767200,14,3,24.0,0.0,537,264314.0,1,1,Eco-Friendly Tires and Wheels Teal #409,DMR,8.0,3.3,4.7,Tires and Wheels
4,2561668,2022-01-03,2767200,16,10,99.0,0.0,474,626349.0,1,1,Eco-Friendly Tires and Wheels Teal #409,DMR,8.0,3.3,4.7,Tires and Wheels


In [25]:
vendedor_indicadores = pd.merge(resultado_final, vendedor, on='codigo_vendedor')

In [26]:
vendedor_indicadores

,codigo_contrato,data_venda,codigo_produto,filial_venda,quantidade,valor_monetario_total,valor_desconto,codigo_vendedor,codigo_cliente,mes,semana,descricaoproduto,codigo_familia,preco_venda_unitario,custo_produto_unitario,lucro,descricaofamilia,nome_vendedor
0,2561471,2022-01-03,2767200,14,5,49.5,0.0,589,325938.0,1,1,Eco-Friendly Tires and Wheels Teal #409,DMR,8.0,3.30,4.70,Tires and Wheels,Eric Dinardo
1,2563189,2022-01-04,2767200,14,2,19.8,0.0,589,388361.0,1,1,Eco-Friendly Tires and Wheels Teal #409,DMR,8.0,3.30,4.70,Tires and Wheels,Eric Dinardo
2,2564091,2022-01-05,2767200,14,3,29.7,0.0,589,715185.0,1,1,Eco-Friendly Tires and Wheels Teal #409,DMR,8.0,3.30,4.70,Tires and Wheels,Eric Dinardo
3,2564116,2022-01-05,2767200,14,7,69.3,0.0,589,684089.0,1,1,Eco-Friendly Tires and Wheels Teal #409,DMR,8.0,3.30,4.70,Tires and Wheels,Eric Dinardo
4,2564832,2022-01-06,2767200,14,2,19.8,0.0,589,250021.0,1,1,Eco-Friendly Tires and Wheels Teal #409,DMR,8.0,3.30,4.70,Tires and Wheels,Eric Dinardo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196104,2768404,2022-09-08,2954000,1,1,730.0,0.0,645,802671.0,9,36,Innovative Musical Instruments Orange #392,CRM,868.0,513.13,354.87,Musical Instruments,Sherry Jeffers
196105,2768432,2022-09-08,2954000,1,1,730.0,0.0,645,802567.0,9,36,Innovative Musical Instruments Orange #392,CRM,868.0,513.13,354.87,Musical Instruments,Sherry Jeffers
196106,2787466,2022-10-04,2981700,1,1,1800.0,0.0,645,642538.0,10,40,Elegant Musical Instruments Black #141,CRM,1759.0,1099.01,659.99,Musical Instruments,Sherry Jeffers
196107,2707859,2022-06-21,2389700,1,1,6745.0,0.0,645,716209.0,6,25,Useful Outdoor Gear Yellow #614,0333,6999.0,4869.59,2129.41,Outdoor Gear,Sherry Jeffers


In [27]:
maior_faturamento_vendedor = vendedor_indicadores.groupby('nome_vendedor')['valor_monetario_total'].sum().reset_index().sort_values(by='valor_monetario_total', ascending=False)
maior_faturamento_vendedor.head(10)

,nome_vendedor,valor_monetario_total
1,Alice Hagan,858503.31
20,Herbert Lindley,817289.40
7,Carol Osdoba,744011.63
34,Mary Sasser,730193.24
17,Eric Dinardo,659222.23
31,Mark Davis,650851.63
23,James Shearer,634378.27
2,Amy Caldeira,621382.50
41,Richard Swift,605482.29
38,Nancy Crockett,600331.94


In [28]:
maior_faturamento_vendedor['nome_vendedor'][0]

'Al Turner'

In [29]:
maior_faturamento_vendedor['valor_monetario_total'][0]

440473.63

In [30]:
melhor_volume_vendas = vendedor_indicadores.groupby('nome_vendedor')['valor_monetario_total'].count().reset_index().sort_values(by='valor_monetario_total', ascending=False)
melhor_volume_vendas.head(20)

,nome_vendedor,valor_monetario_total
20,Herbert Lindley,5693
7,Carol Osdoba,5618
23,James Shearer,5510
17,Eric Dinardo,5372
24,Joan Krug,5228
1,Alice Hagan,5051
31,Mark Davis,4787
10,Dave Soto,4697
0,Al Turner,4548
44,Shaun Seidel,4538


In [31]:
melhor_volume_vendas['nome_vendedor'][5]

'Bill Armstrong'

In [32]:
melhor_volume_vendas['valor_monetario_total'][5]

3318

In [33]:
ticket_medio = vendedor_indicadores.groupby('nome_vendedor')['valor_monetario_total'].mean().reset_index().sort_values(by='valor_monetario_total', ascending=False)
round(ticket_medio).head(20)

,nome_vendedor,valor_monetario_total
1,Alice Hagan,170.0
9,Cheryl Ingraham,169.0
33,Marlin Hammett,164.0
34,Mary Sasser,161.0
39,Pamela Herrera,157.0
38,Nancy Crockett,154.0
41,Richard Swift,149.0
4,Barry Corona,149.0
40,Patricia Francis,148.0
12,Donnie Vernon,148.0


In [34]:
round(ticket_medio['valor_monetario_total'])[36]

141.0

In [35]:

# Convertendo as séries em DataFrame e redefinindo o índice
maior_faturamento_vendedor = maior_faturamento_vendedor.reset_index()
melhor_volume_vendas = melhor_volume_vendas.reset_index()

# Adicionando colunas de ranking para cada indicador
maior_faturamento_vendedor['rank_faturamento'] = maior_faturamento_vendedor['valor_monetario_total'].rank(ascending=False)
melhor_volume_vendas['rank_volume_vendas'] = melhor_volume_vendas['valor_monetario_total'].rank(ascending=False)
ticket_medio['rank_ticket_medio'] = ticket_medio['valor_monetario_total'].rank(ascending=False)

# Definindo os pesos para cada indicador
peso_faturamento = 0.4
peso_volume_vendas = 0.3
peso_ticket_medio = 0.3

# Preparando o DataFrame para a pontuação combinada
df_pontuacao = maior_faturamento_vendedor[['nome_vendedor', 'rank_faturamento']].copy()
df_pontuacao = df_pontuacao.merge(melhor_volume_vendas[['nome_vendedor', 'rank_volume_vendas']], on='nome_vendedor')
df_pontuacao = df_pontuacao.merge(ticket_medio[['nome_vendedor', 'rank_ticket_medio']], on='nome_vendedor')

# Calculando a pontuação combinada
df_pontuacao['pontuacao'] = (df_pontuacao['rank_faturamento'] * peso_faturamento) + \
                            (df_pontuacao['rank_volume_vendas'] * peso_volume_vendas) + \
                            (df_pontuacao['rank_ticket_medio'] * peso_ticket_medio)

# Ordenando os vendedores pela pontuação final
df_pontuacao.sort_values(by='pontuacao', ascending=True, inplace=True)

# Selecionando os top 5 vendedores
top_5_vendedores = df_pontuacao.head(5)

top_5_vendedores
 

,nome_vendedor,rank_faturamento,rank_volume_vendas,rank_ticket_medio,pontuacao
0,Alice Hagan,1.0,6.0,1.0,2.5
1,Herbert Lindley,2.0,1.0,12.0,4.7
3,Mary Sasser,4.0,12.0,4.0,6.4
2,Carol Osdoba,3.0,2.0,18.0,7.2
5,Mark Davis,6.0,7.0,16.0,9.3


In [36]:
# Criação do gráfico
fig4 = go.Figure()

fig4.add_trace(go.Indicator(
    mode='number',
    value=maior_faturamento_vendedor['valor_monetario_total'][0],
    number={'valueformat': ".2s", 'prefix': "R$"},
    title={
        'text': f"Melhor Vendedor(a) - {maior_faturamento_vendedor['nome_vendedor'][0]}<br><span style='font-size:100%'>Faturamento Anual</span>"
    }
))

# Adição de anotações com informações adicionais
fig4.add_annotation(
    x=0.5,
    y=0.2,
    xref='paper',
    yref='paper',
    text=(
        f"Volume de Vendas: {melhor_volume_vendas['valor_monetario_total'][5]} vendas<br>"
        f"Ticket Médio: R$ {round(ticket_medio['valor_monetario_total'][1], 2):,} reais"
    ),
    showarrow=False
)

# Configurações do layout
fig4.update_layout(height=450)

# Exibição do gráfico
fig4.show()

In [37]:
# Criação do gráfico
fig5 = go.Figure()

fig5.add_trace(go.Indicator(
    mode='number',
    value=maior_faturamento_vendedor['valor_monetario_total'][1],
    number={'valueformat': ".2s", 'prefix': "R$"},
    title={
        'text': f"2º Melhor Vendedor(a) - {maior_faturamento_vendedor['nome_vendedor'][1]}<br><span style='font-size:100%'>Faturamento Anual</span>"
    }
))

# Adição de anotações com informações adicionais
fig5.add_annotation(
    x=0.5,
    y=0.2,
    xref='paper',
    yref='paper',
    text=(
        f"Volume de Vendas: {melhor_volume_vendas['valor_monetario_total'][0]} vendas<br>"
        f"Ticket Médio: R$ {round(ticket_medio['valor_monetario_total'][20], 2):,} reais"
    ),
    showarrow=False
)

# Configurações do layout
fig5.update_layout(height=450)

# Exibição do gráfico
fig5.show()

In [38]:
# Criação do gráfico
fig6 = go.Figure()

fig6.add_trace(go.Indicator(
    mode='number',
    value=maior_faturamento_vendedor['valor_monetario_total'][3],
    number={'valueformat': ".2s", 'prefix': "R$"},
    title={
        'text': f"3º Melhor Vendedor(a) - {maior_faturamento_vendedor['nome_vendedor'][3]}<br><span style='font-size:100%'>Faturamento Anual</span>"
    }
))

# Adição de anotações com informações adicionais
fig6.add_annotation(
    x=0.5,
    y=0.2,
    xref='paper',
    yref='paper',
    text=(
        f"Volume de Vendas: {melhor_volume_vendas['valor_monetario_total'][11]} vendas<br>"
        f"Ticket Médio: R$ {round(ticket_medio['valor_monetario_total'][34], 2):,} reais"
    ),
    showarrow=False
)

# Configurações do layout
fig6.update_layout(height=450)

# Exibição do gráfico
fig6.show()

In [39]:
# Criação do gráfico
fig7 = go.Figure()

fig7.add_trace(go.Indicator(
    mode='number',
    value=maior_faturamento_vendedor['valor_monetario_total'][2],
    number={'valueformat': ".2s", 'prefix': "R$"},
    title={
        'text': f" 4º Melhor Vendedor(a) - {maior_faturamento_vendedor['nome_vendedor'][2]}<br><span style='font-size:100%'>Faturamento Anual</span>"
    }
))

# Adição de anotações com informações adicionais
fig7.add_annotation(
    x=0.5,
    y=0.2,
    xref='paper',
    yref='paper',
    text=(
        f"Volume de Vendas: {melhor_volume_vendas['valor_monetario_total'][1]} vendas<br>"
        f"Ticket Médio: R$ {round(ticket_medio['valor_monetario_total'][7], 2):,} reais"
    ),
    showarrow=False
)

# Configurações do layout
fig7.update_layout(height=450)

# Exibição do gráfico
fig7.show()

In [40]:
# Criação do gráfico
fig8 = go.Figure()

fig8.add_trace(go.Indicator(
    mode='number',
    value=maior_faturamento_vendedor['valor_monetario_total'][5],
    number={'valueformat': ".2s", 'prefix': "R$"},
    title={
        'text': f" 5º Melhor Vendedor(a) - {maior_faturamento_vendedor['nome_vendedor'][5]}<br><span style='font-size:100%'>Faturamento Anual</span>"
    }
))
# Adição de anotações com informações adicionais
fig8.add_annotation(
    x=0.5,
    y=0.2,
    xref='paper',
    yref='paper',
    text=(
        f"Volume de Vendas: {melhor_volume_vendas['valor_monetario_total'][6]} vendas<br>"
        f"Ticket Médio: R$ {round(ticket_medio['valor_monetario_total'][31], 2):,} reais"
    ),
    showarrow=False
)

# Configurações do layout
fig8.update_layout(height=450)

# Exibição do gráfico
fig8.show()

### Pergunta 2: O departamento comercial planeja fazer campanhas de marketing para impulsionar a venda de certos produtos. Isto posto, ele gostaria que você indicasse até cinco famílias de produtos para eles fazerem uma campanha no mês de Dezembro.
> A. Quais famílias você indicaria e por quê?

> B. Dica: Considere as tendências de mercado. Avalie o histórico de vendas das famílias de produtos.


In [41]:
# Substituindo 'Power Tools' por 'Tools'
resultado_final['descricaofamilia'] = resultado_final['descricaofamilia'].replace('Power Tools', 'Tools')

In [42]:
agrupado_por_familias = resultado_final.groupby('descricaofamilia')['valor_monetario_total'].sum().sort_values(ascending=False)
agrupado_por_familias.head(6)

descricaofamilia
Home Appliances    2724705.71
Lighting           1533393.14
Electronics        1239529.95
Tools              1147909.57
Video Games        1074433.35
Office Supplies     826696.31
Name: valor_monetario_total, dtype: float64

In [43]:
# Supondo que 'agrupado_por_familias' seja o seu DataFrame e que ele já esteja agrupado e ordenado corretamente
# Certifique-se de que 'agrupado_por_familias' é um DataFrame com uma coluna para famílias e outra para valores

# Criar um gráfico de barras
fig9 = px.bar(agrupado_por_familias.reset_index(), 
             x='descricaofamilia',  # Usar o nome da coluna para as famílias de produtos
             y='valor_monetario_total',  # Coluna para o valor monetário total
             title="Faturamento Total por Família de Produtos",
             labels={'descricaofamilia': 'Família de Produtos', 'valor_monetario_total': 'Faturamento Total (R$)'})

# Ajustar o layout para aumentar a largura
fig9.update_layout(width=1100, height=600)  # Você pode ajustar estes valores conforme necessário

# Mostrar o gráfico
fig9.show()

In [44]:
# Filtrando os dados para incluir apenas os meses de junho a dezembro
dados_filtrados = resultado_final[resultado_final['mes'].between(6, 12)]

# Agrupando por família e somando o valor monetário total
agrupado_por_familias1 = dados_filtrados.groupby('descricaofamilia')['valor_monetario_total'].sum().sort_values(ascending=False)

# Exibindo os cinco primeiros resultados
top_familias = agrupado_por_familias1.head()
print(top_familias)

descricaofamilia
Home Appliances    1706127.99
Lighting           1022784.94
Electronics         811649.76
Tools               732564.54
Office Supplies     499207.70
Name: valor_monetario_total, dtype: float64


In [45]:
fig10 = px.bar(top_familias.reset_index(), 
             x='valor_monetario_total',  # Coluna para o valor monetário total
             y='descricaofamilia',  # Usar o nome da coluna para as famílias de produtos
             orientation='h',  # Definir orientação horizontal
             title="Top 5 Famílias de Tendência para Campanha de Dezembro",
             labels={'descricaofamilia': 'Família de Produtos', 'valor_monetario_total': 'Faturamento Total (R$)'})

# Ajustar o layout para definir a largura e a altura
fig10.update_layout(width=800, height=400)  # Ajuste estes valores conforme necessário

# Mostrar o gráfico
fig10.show()

In [46]:
# Agrupe por mês e vendedor e calcule a soma do valor monetário total para cada grupo
melhores_familia_por_mes = resultado_final.groupby(['mes', 'descricaofamilia'])['valor_monetario_total'].sum().reset_index()

# Ordena o DataFrame pelo valor monetário total em ordem decrescente para cada mês
melhores_familia_por_mes = melhores_familia_por_mes.sort_values(by=['mes', 'valor_monetario_total'], ascending=[True, False])

melhores_familia_por_mes

,mes,descricaofamilia,valor_monetario_total
61,1,Video Games,214284.76
57,1,Tires and Wheels,207740.43
29,1,Home Appliances,140994.38
58,1,Tools,82453.56
18,1,Electronics,75319.19
...,...,...,...
731,12,Beverages,10905.17
779,12,Sports Equipment,10430.48
736,12,Clothing,6642.18
747,12,Furniture,5872.25


In [47]:
# Lista para armazenar os top 5 de cada mês
top_5_familia_por_mes = []

# Iterar através de cada mês e selecionar os top 5
for mes in range(1, 13):
    top_5_do_mes = melhores_familia_por_mes[melhores_familia_por_mes['mes'] == mes].head(5)
    top_5_familia_por_mes.append(top_5_do_mes)

# Concatenando todos os DataFrames dos meses
top_5_familias_ano = pd.concat(top_5_familia_por_mes, ignore_index=True)

# Contando a frequência de cada família
frequencia_familias = top_5_familias_ano['descricaofamilia'].value_counts()

# Selecionando as 5 famílias mais frequentes
cinco_melhores_familias = frequencia_familias.head(6).index.tolist()

# Calculando a média de faturamento para cada uma das 5 melhores famílias
medias_faturamento = top_5_familias_ano.groupby('descricaofamilia')['valor_monetario_total'].mean()

# Exibindo as famílias e seus faturamentos médios
print("Faturamento médio das 5 famílias com melhores faturamentos durante os meses:")
for familia in cinco_melhores_familias:
    media_familia = medias_faturamento[familia]
    print(f"{familia}: {media_familia:.2f}")

Faturamento médio das 5 famílias com melhores faturamentos durante os meses:
Home Appliances: 227058.81
Tools: 95659.13
Electronics: 103294.16
Lighting: 139360.65
Video Games: 125695.11
Office Supplies: 78864.01


In [48]:
# Criando um dicionário para armazenar os meses das top 5 famílias
meses_das_familias = {familia: [] for familia in cinco_melhores_familias}

# Iterando pelos DataFrames mensais e verificando a presença das famílias
for mes, df in enumerate(top_5_familia_por_mes, start=1):
    for familia in cinco_melhores_familias:
        if familia in df['descricaofamilia'].values:
            meses_das_familias[familia].append(mes)

# Exibindo os meses de desempenho top para cada família
print("Meses de melhor desempenho para as 5 melhores famílias:")
for familia, meses in meses_das_familias.items():
    meses_formatados = ', '.join(map(str, meses))
    print(f"{familia}: {meses_formatados}")

Meses de melhor desempenho para as 5 melhores famílias:
Home Appliances: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12
Tools: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12
Electronics: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12
Lighting: 3, 4, 5, 6, 7, 8, 9, 10, 11, 12
Video Games: 1, 2, 3, 4, 5, 6
Office Supplies: 7, 8, 11


#### Pergunta 4:
> Clientes Premium: Clientes que compram todos os meses.

> Clientes Recorrentes: Clientes que realizaram mais de 1 compra.

> Faturamento por mês por cada filial.

In [49]:
vendas['codigo_cliente'].head()

0    325938.0
1    325938.0
2    325938.0
3    608986.0
4    689748.0
Name: codigo_cliente, dtype: float64

In [50]:
# Calcule o número total de meses
num_meses_total = len(vendas['mes'].unique())
num_meses_total

12

In [51]:
# Agrupe por cliente e conte o número de meses únicos
clientes_por_mes = vendas.groupby('codigo_cliente')['mes'].nunique().reset_index()

# Filtra os clientes que têm o número de meses únicos igual ao número total de meses
clientes_premium = clientes_por_mes[clientes_por_mes['mes'] == num_meses_total]['codigo_cliente']
clientes_premium
len(clientes_premium)

80

In [52]:

fig11 = go.Figure()
fig11.add_trace(go.Indicator(
    mode='number',
    title = {
        "text": f"<span style='font-size:100%'>Clientes Frequentes</span><br><span style='font-size:50%'>Clientes que realizaram compras todos os meses</span>"
    },
    value = len(clientes_premium)
))

# Outras configurações do gráfico, se necessário...

fig11.show()

In [53]:
todos_clientes = vendas['codigo_cliente'].nunique()
todos_clientes

61601

In [54]:
contagem_retencao_cpf = vendas['codigo_cliente'].value_counts()
len(contagem_retencao_cpf)

61601

In [55]:
clientes_recorrentes = contagem_retencao_cpf[contagem_retencao_cpf > 1]
clientes_recorrentes

341769.0    2067
796513.0     847
782568.0     561
727915.0     480
341770.0     460
            ... 
778994.0       2
800026.0       2
614851.0       2
783856.0       2
299716.0       2
Name: codigo_cliente, Length: 32734, dtype: int64

In [56]:
# Calcular o número de clientes não recorrentes
clientes_nao_recorrentes = len(contagem_retencao_cpf)- len(clientes_recorrentes)

# Dados para o gráfico de pizza
labels = ['Clientes Recorrentes', 'Clientes Não Recorrentes']
valores = [len(clientes_recorrentes), clientes_nao_recorrentes]
cores = ['lightcoral', 'lightskyblue']

# Criar um objeto go.Figure
fig12 = go.Figure()

# Adicionar o gráfico de pizza
fig12.add_trace(go.Pie(labels=labels, values=valores, textinfo='percent', 
                     marker=dict(colors=cores, line=dict(color='#000000', width=5)),
                     hole=0.5))

# Atualizar layout
fig12.update_layout(
    title='Proporção de Clientes Recorrentes',
    title_x=0.5,
    showlegend=True,
    legend=dict(x=0.6, y=0.2),
)

# Exibir o gráfico
fig12.show()

In [57]:
vendas['filial_venda'].value_counts()

vendas['filial_venda'].nunique()

filial = vendas.groupby(['filial_venda', 'mes'])['valor_monetario_total'].sum().reset_index()
filial

,filial_venda,mes,valor_monetario_total
0,1,1,301669.53
1,1,2,263357.89
2,1,3,287860.15
3,1,4,308765.83
4,1,5,303438.29
...,...,...,...
79,17,8,432196.22
80,17,9,392444.35
81,17,10,388653.57
82,17,11,415019.55


In [58]:
# Crie uma tabela pivô para facilitar o plot
tabela_pivot = filial.pivot(index='mes', columns='filial_venda', values='valor_monetario_total')
tabela_pivot

filial_venda,1,8,10,11,14,16,17
mes,,,,,,,
1,301669.53,165336.66,201015.87,107292.10,520727.29,248755.58,400872.14
2,263357.89,164648.78,179277.83,188195.14,410345.09,224494.40,392473.76
3,287860.15,156746.38,170313.58,206304.15,422706.11,236711.44,396193.77
4,308765.83,205342.97,209646.17,248466.37,408526.21,254350.46,341856.10
5,303438.29,171660.99,256485.65,168458.16,502117.70,281208.69,431179.05
6,255392.14,172493.87,321536.28,176395.78,421681.45,258215.84,329698.95
7,247071.40,230267.90,380608.04,206303.36,420014.91,271895.55,353258.71
8,294574.45,313579.24,430096.41,241740.42,492278.74,253997.39,432196.22
9,262185.48,219244.24,365360.30,222241.91,435119.01,259405.48,392444.35


In [59]:
# Crie um objeto go.Figure
fig13 = go.Figure()

# Adicione barras empilhadas ao gráfico
for coluna in tabela_pivot.columns:
    fig13.add_trace(go.Bar(
        x=tabela_pivot.index,
        y=tabela_pivot[coluna],
        name=coluna
    ))

# Atualize o layout do gráfico
fig13.update_layout(
    title='Valor Monetário por Filial e Mês',
    xaxis=dict(title='Mês'),
    yaxis=dict(title='Valor Monetário Total'),
    barmode='stack'
)
# Adicionando rótulos ao eixo x
fig13.update_xaxes(tickvals=agrupado_por_mes['mes'], ticktext=['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Aug', 'Set', 'Out', 'Nov', 'Dez'])
# Exiba o gráfico
fig13.show()

## Taxa Percentual de Crescimento Mensal -MoM
Taxa Percentual de Crescimento Mensal = Faturamento Mensal/Faturamento Mensal Anterior * 100

In [60]:

# Certifique-se de que os dados estão ordenados por mês
vendas.sort_values('mes', inplace=True)

# Calculando o faturamento total por mês
faturamento_mensal = vendas.groupby('mes')['valor_monetario_total'].sum()

# Calculando a Taxa Percentual de Crescimento Mensal
tpcm = faturamento_mensal.pct_change() * 100

# Substituindo NaN por 0 para o primeiro mês
tpcm = tpcm.fillna(0)

# Exibindo o resultado
print(tpcm)

mes
1      0.000000
2     -6.315374
3      2.964829
4      5.334433
5      6.959920
6     -8.471511
7      8.990610
8     16.546872
9    -12.302895
10    -0.749520
11    -9.006421
12    17.810432
Name: valor_monetario_total, dtype: float64


In [61]:
# Supondo que você tenha uma lista de meses correspondente
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

# Criando o gráfico de barras com a cor das barras alterada para violeta
fig14 = go.Figure(data=[
    go.Bar(x=meses, y=tpcm, marker_color='violet')  # Alterado para violeta
])

# Adicionando títulos e rótulos
fig14.update_layout(
    title='Taxa Percentual de Crescimento Mensal do Faturamento',
    xaxis_title='Mês',
    yaxis_title='Taxa de Crescimento (%)',
    template='plotly_white'  # Mantém o fundo branco do gráfico
)

# Mostrando o gráfico
fig14.show()